In [1]:
using ForwardDiff

In [2]:
import ForwardDiff

In [3]:
# function f(x::Vector{T}) where T <: Number
function f(x)
    a = x[1]
    b = x[2]
    x1 = a+b
    x2 = a*exp(b)
    return [x1,x2]
end

f (generic function with 1 method)

In [4]:
vec = [1, 2]
f(vec)

2-element Vector{Float64}:
 3.0
 7.38905609893065

In [5]:
ForwardDiff.jacobian(f, vec)

2×2 Matrix{Float64}:
 1.0      1.0
 7.38906  7.38906

In [6]:
model = (α = 0.3, β = 0.96,  ρ = 0.9, γ = 4.0, δ = 0.1,)

(α = 0.3, β = 0.96, ρ = 0.9, γ = 4.0, δ = 0.1)

In [7]:
typeof(model)

@NamedTuple{α::Float64, β::Float64, ρ::Float64, γ::Float64, δ::Float64}

In [8]:
function transition(z, k, i, p)
    (; δ, ρ, α) = p

    k = (1-δ)*k + i

    z = ρ*z

    return z, k
end

transition (generic function with 1 method)

In [9]:
transition(1, 2, 3, model)

(0.9, 4.8)

In [10]:
function arbitrage(z, k, i, Z, K, I, p)
    (; δ, ρ, α, γ, β) = p

    c = exp(z)*k^α - i
    C = exp(Z)*K^α - I

    resid = β*( (C/c)^(-γ) * ( exp(Z) * α*K^(α-1) + (1-δ) ) ) - 1

    return resid
end

arbitrage (generic function with 1 method)

In [11]:
arbitrage(1, 2, 3, 4, 5, 6, model)

-0.9999999981417501

In [12]:
function transition(s::Vector{T},x::Vector{U},model) where T<:Number where U<:Number

    res = transition(s[1], s[2], x[1], model)

    return [res...]
end

transition (generic function with 2 methods)

In [13]:
svec = [1, 2]
xvec = [1]
transition(svec, xvec, model)

2-element Vector{Float64}:
 0.9
 2.8

In [14]:
function arbitrage(s::Vector{}, x::Vector{}, S::Vector{}, X::Vector{}, p)

    resid = arbitrage(s[1], s[2], x[1], S[1], S[2], X[1], p)
    
    return [resid]
end 

arbitrage (generic function with 2 methods)

In [15]:
arbitrage(svec, xvec, svec, xvec, model)

1-element Vector{Float64}:
 0.34591003812862753

In [16]:
function steady_state(p)

    (; δ, ρ, α, γ, β) = p

    kss = ( (1/β - (1-δ)) / α )^(1/(α-1))
    iss = δ*kss
    zss = 0.0
    css = kss^α - iss

    return (;
    s = [zss, kss], 
    x = [iss],
    )
    
end

steady_state (generic function with 1 method)

In [17]:
ss = steady_state(model)

(s = [0.0, 2.920822149964071], x = [0.29208221499640713])

In [18]:
r1 = arbitrage(ss.s, ss.x, ss.s, ss.x, model)

1-element Vector{Float64}:
 0.0

In [19]:
r2 = transition(ss.s, ss.x, model) - ss.s

2-element Vector{Float64}:
 0.0
 0.0

In [20]:
maximum(abs, [r1; r2]) 

0.0

In [21]:
import ForwardDiff: jacobian

In [22]:
A = jacobian(u->arbitrage(u, ss.x, ss.s, ss.x, model), ss.s)
B = jacobian(u->arbitrage(ss.s, u, ss.s, ss.x, model), ss.x)
C = jacobian(u->arbitrage(ss.s, ss.x, u, ss.x, model), ss.s)
D = jacobian(u->arbitrage(ss.s, ss.x, ss.s, u, model), ss.x)

1×1 Matrix{Float64}:
 3.679193085018409

In [23]:
E = jacobian(u->transition(u, ss.x, model), ss.s)
F = jacobian(u->transition(ss.s, u, model), ss.x)

2×1 Matrix{Float64}:
 0.0
 1.0

In [26]:
function first_order_model(s::Vector, x::Vector, model)
    
    (; δ, ρ, α, γ, β) = model

    A = jacobian(u->arbitrage(u, ss.x, ss.s, ss.x, model), ss.s)
    B = jacobian(u->arbitrage(ss.s, u, ss.s, ss.x, model), ss.x)
    C = jacobian(u->arbitrage(ss.s, ss.x, u, ss.x, model), ss.s)
    D = jacobian(u->arbitrage(ss.s, ss.x, ss.s, u, model), ss.x)
    E = jacobian(u->transition(u, ss.x, model), ss.s)
    F = jacobian(u->transition(ss.s, u, model), ss.x)

    return (; A, B, C, D, E, F)
end

first_order_model (generic function with 1 method)

In [27]:
fo = first_order_model(ss.s, ss.x, model)

(A = [5.074626865671641 0.5212190203776083], B = [-3.679193085018409;;], C = [-4.938626865671641 -0.5538125831185546], D = [3.679193085018409;;], E = [0.9 0.0; 0.0 0.9], F = [0.0; 1.0;;])

In [61]:
function residual(X, M)

    (; A, B, C, D, E, F) = M

    resid = (A+B*X) + (C+D*X) * (E+F*X) 

    return resid
end

residual (generic function with 1 method)

In [66]:
# x = Xs 
# dim of X: nx,ns
X0 = [1.0 2.0]

1×2 Matrix{Float64}:
 1.0  2.0

In [ ]:
residual(X0, fo)

0.14149778291769888

In [103]:
function T(X, M)
    
    (; A, B, C, D, E, F) = M

    # X is decision rule tmr, Xtilde in F(X, Xtilde)
    Xprime = - inv(B + (C+D*X) * F) * (A + (C+D*X) * E)
    
    return Xprime # decision rule today
end

T (generic function with 1 method)

In [104]:
T(X0,fo)

1×2 Matrix{Float64}:
 0.435186  0.195785

In [112]:
function linear_time_iteration(X, M; N=1000, τ_ϵ=1e-8, τ_η=1e-8)
     
    global Xstar = X

    η = []
    ϵ = []
    for n in 1:N

        Xprime = T(X,M)
        push!(η, maximum(abs, Xprime - X))
        push!(ϵ, maximum(abs, residual(Xprime,M)))

        if ϵ[end] < τ_ϵ || η[end] < τ_η
            Xstar = Xprime
            break
        end
        X = Xprime
    end

    return (; Xstar, η, ϵ)
end

linear_time_iteration (generic function with 1 method)

In [113]:
X0 = rand(1,2)

1×2 Matrix{Float64}:
 0.486798  0.23388

In [116]:
results = linear_time_iteration(X0, fo)

(Xstar = [0.7686740081802743 0.027809726836878046], η = Any[0.17792368810589188, 0.17699745129099786, 0.17774002848330372, 0.18073228045853607, 0.18683690672652964, 0.19737854704434343, 0.21447584190895608, 0.2416923085045546, 0.2854116549625121, 0.35803326375120204  …  4.833978828333585e-8, 4.001382314378077e-8, 3.312018559409324e-8, 2.7412814462834945e-8, 2.2687851264358017e-8, 1.8776414867360813e-8, 1.553861195269235e-8, 1.2858569209406312e-8, 1.0640317293919566e-8, 8.80437689421143e-9], ϵ = Any[0.595292209777551, 0.5957493722238905, 0.6031837683135651, 0.6201760880722499, 0.6506205027480366, 0.7006112351387634, 0.7801211388292124, 0.9063976592010436, 1.1114210768951507, 1.4600950814254325  …  1.6528463264720017e-7, 1.3680916399749776e-7, 1.1323379256467092e-7, 9.371644216216168e-8, 7.755951747512313e-8, 6.4185162518271e-8, 5.3114741493942574e-8, 4.395183461625152e-8, 3.6368136502318293e-8, 3.0091773517426645e-8])

In [118]:
results.ϵ

113-element Vector{Any}:
 0.595292209777551
 0.5957493722238905
 0.6031837683135651
 0.6201760880722499
 0.6506205027480366
 0.7006112351387634
 0.7801211388292124
 0.9063976592010436
 1.1114210768951507
 1.4600950814254325
 ⋮
 1.3680916399749776e-7
 1.1323379256467092e-7
 9.371644216216168e-8
 7.755951747512313e-8
 6.4185162518271e-8
 5.3114741493942574e-8
 4.395183461625152e-8
 3.6368136502318293e-8
 3.0091773517426645e-8

In [120]:
residual(results.Xstar, fo)

1×2 Matrix{Float64}:
 3.00918e-8  1.13228e-9